In [1]:
import ee
import geemap
import datetime
import numpy as np
import pandas as pd
from dateutil import tz

In [ ]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

modis_date_start = ee.Date("2001-01-01") # в 2001 начал предоставлять тепловую информацию Terra
landsat_date_start = ee.Date("1986-03-16") # с 1986 начал предоставлять тепловую информацию Landsat-5
landsat_start_year = landsat_date_start.get("year").getInfo()
modis_start_year = modis_date_start.get("year").getInfo()
date_end = datetime.datetime.now()
end_year = date_end.date().year
openwater_month_start = 6 # июнь
openwater_month_end = 11 # ноябрь

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

(node:8778) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!


# LANDSAT

In [ ]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [4]:
use_ndvi = True

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, landsat_date_start, date_end, baikal_shape, use_ndvi)#.filter(ee.Filter.lt('CLOUD_COVER', 30))
   return coll.select(['LST'])

In [5]:
landsats_collection = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L7'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L5'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L5'))

In [6]:
def get_average_lst_landsat(landsat_collection: ee.ImageCollection, start_time, end_time):
   temp = landsat_collection.filterDate(start_time, end_time).mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return result_dict['LST']

def add_lst(image, d):
   temp_time = image.getString('DATE_ACQUIRED').cat(" ").cat(image.getString('SCENE_CENTER_TIME')).getInfo()
   time = pd.to_datetime(temp_time, format='%Y-%m-%d %H:%M:%S.%fZ')
   mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000).getInfo()['LST']
   d[time] = mean
   return d

def get_average_lst_landsat(landsat_collection: ee.ImageCollection):
   result = {}
   coll_size = landsat_collection.size().getInfo()
   if coll_size == 0:
      return result
   landsat_list = landsat_collection.toList(coll_size)
   for i in range(coll_size):
      image = ee.Image(landsat_list.get(i))
      temp_time = image.getString('DATE_ACQUIRED').cat(" ").cat(image.getString('SCENE_CENTER_TIME')).getInfo()
      time = pd.to_datetime(temp_time, format='%Y-%m-%d %H:%M:%S.%fZ')
      mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000).getInfo()['LST']
      result[time] = mean
   return result

In [ ]:
all_years_landsat = {}
for year in range(1988, 2025):
   year_coll = landsats_collection.filterDate(ee.Date(str(year)), ee.Date(str(year+1)))
   result = get_average_lst_landsat(year_coll)
   print(year, result)
   all_years_landsat[year] = result
   
all_years_landsat

1984 {}
1985 {}
1986 {Timestamp('1986-07-13 03:01:24.924031'): 16.27339768998559, Timestamp('1986-07-29 03:00:51.344056'): 16.573165869364228, Timestamp('1986-11-02 02:57:32.929000'): -13.227672344186969, Timestamp('1986-07-13 03:01:48.785013'): 12.091803148713247, Timestamp('1986-07-29 03:01:15.206081'): 15.148326739520522, Timestamp('1986-11-02 02:57:56.660081'): -18.596356014020028, Timestamp('1986-07-13 03:02:12.788081'): 7.4542387361959594, Timestamp('1986-07-29 03:01:39.212006'): 13.97481009402597, Timestamp('1986-11-02 02:58:20.821050'): -26.57226152621019, Timestamp('1986-11-18 02:57:43.266088'): -3.4592302068156173, Timestamp('1986-06-11 03:03:39.572019'): None, Timestamp('1986-07-13 03:02:36.506075'): None, Timestamp('1986-11-02 02:58:44.553038'): None, Timestamp('1986-11-18 02:58:07'): 2.5347998448313156, Timestamp('1986-06-02 03:08:55.313094'): -3.5995585078764716, Timestamp('1986-06-18 03:08:25.396050'): 6.847358187443933, Timestamp('1986-11-09 03:03:27.547094'): 0.3093797

KeyboardInterrupt: 

In [ ]:

1986 {Timestamp('1986-07-13 03:01:24.924031'): 16.27339768998559, Timestamp('1986-07-29 03:00:51.344056'): 16.573165869364228, Timestamp('1986-11-02 02:57:32.929000'): -13.227672344186969, Timestamp('1986-07-13 03:01:48.785013'): 12.091803148713247, Timestamp('1986-07-29 03:01:15.206081'): 15.148326739520522, Timestamp('1986-11-02 02:57:56.660081'): -18.596356014020028, Timestamp('1986-07-13 03:02:12.788081'): 7.4542387361959594, Timestamp('1986-07-29 03:01:39.212006'): 13.97481009402597, Timestamp('1986-11-02 02:58:20.821050'): -26.57226152621019, Timestamp('1986-11-18 02:57:43.266088'): -3.4592302068156173, Timestamp('1986-06-11 03:03:39.572019'): None, Timestamp('1986-07-13 03:02:36.506075'): None, Timestamp('1986-11-02 02:58:44.553038'): None, Timestamp('1986-11-18 02:58:07'): 2.5347998448313156, Timestamp('1986-06-02 03:08:55.313094'): -3.5995585078764716, Timestamp('1986-06-18 03:08:25.396050'): 6.847358187443933, Timestamp('1986-11-09 03:03:27.547094'): 0.30937971240183865, Timestamp('1986-06-02 03:09:19.171038'): -0.5565752948732023, Timestamp('1986-06-18 03:08:49.255038'): 2.6156504669638636, Timestamp('1986-11-09 03:03:51.423019'): -2.229586178007629, Timestamp('1986-06-18 03:09:13.257025'): 4.177805599950558, Timestamp('1986-11-09 03:04:15.441050'): 0.7690070096073587, Timestamp('1986-06-18 03:09:36.973031'): 7.274843632002761, Timestamp('1986-11-09 03:04:39.173094'): 2.5699043636144236, Timestamp('1986-11-25 03:04:09.901063'): 0.47438310284844043, Timestamp('1986-07-27 03:13:41.122031'): 10.994476261221505, Timestamp('1986-10-31 03:10:22.699063'): None, Timestamp('1986-11-16 03:09:45.756038'): -12.858406287744103, Timestamp('1986-06-09 03:15:41.152044'): None, Timestamp('1986-07-27 03:14:05.127031'): 13.916777285282565, Timestamp('1986-10-31 03:10:46.717025'): -7.904937742310696, Timestamp('1986-06-09 03:16:04.867050'): -0.163580782739512, Timestamp('1986-07-27 03:14:28.846050'): 13.00133318273807, Timestamp('1986-10-31 03:11:10.449000'): -12.16561142818517, Timestamp('1986-11-16 03:10:33.508013'): -5.45060148787239, Timestamp('1986-04-29 03:23:24.899000'): -1.5397001336750642, Timestamp('1986-05-31 03:22:32.065063'): 2.0580334116720014, Timestamp('1986-06-16 03:22:02.443013'): 5.4499889428164145, Timestamp('1986-07-18 03:20:58.664038'): 12.663749845634994, Timestamp('1986-08-03 03:20:24.636063'): 12.29559892737686, Timestamp('1986-10-22 03:17:40.517081'): None, Timestamp('1986-11-07 03:17:05.518075'): 3.9354982805085954, Timestamp('1986-11-23 03:16:28.234056'): -18.621886850289375}
1987 {Timestamp('1987-09-18 03:05:29.994044'): None, Timestamp('1987-11-05 03:06:26.141094'): -4.686794896999162, Timestamp('1987-11-05 03:06:49.913063'): -6.236807808685767, Timestamp('1987-11-05 03:07:14.115000'): -8.311005904337014, Timestamp('1987-11-21 03:07:34.276019'): None, Timestamp('1987-02-13 03:05:28.337000'): -27.54194845941271, Timestamp('1987-09-25 03:11:49.937006'): 9.407315114776772, Timestamp('1987-02-13 03:05:52.077038'): -25.94110678889002, Timestamp('1987-03-17 03:07:05.507063'): None, Timestamp('1987-09-25 03:12:13.846006'): 9.229384806263992, Timestamp('1987-02-13 03:06:16.103081'): -23.57706044407412, Timestamp('1987-03-17 03:07:29.536006'): -11.268020558022181, Timestamp('1987-09-25 03:12:37.898025'): 9.37873750347676, Timestamp('1987-02-13 03:06:39.844031'): -21.956009866166738, Timestamp('1987-03-17 03:07:53.278038'): -12.559052574196516, Timestamp('1987-09-25 03:13:01.521000'): 9.627027131073445, Timestamp('1987-10-11 03:13:20.362025'): 4.588773558784484, Timestamp('1987-02-04 03:11:41.518044'): -13.530588978925058, Timestamp('1987-02-20 03:12:19.441069'): None, Timestamp('1987-04-09 03:14:04.717038'): -4.539291041603977, Timestamp('1987-10-02 03:18:33.208094'): None, Timestamp('1987-02-04 03:12:05.544025'): -13.973192114599925, Timestamp('1987-02-20 03:12:43.468044'): -20.048931007542148, Timestamp('1987-04-09 03:14:28.747094'): -5.358725609738251, Timestamp('1987-10-02 03:18:57.262031'): -3.6621898356175215, Timestamp('1987-11-19 03:19:53.731044'): -11.598809874255549, Timestamp('1987-02-04 03:12:29.284000'): -12.518609215881831, Timestamp('1987-02-20 03:13:07.209025'): None, Timestamp('1987-04-09 03:14:52.492050'): -5.762070732241982, Timestamp('1987-06-28 03:16:58.266025'): 2.3059326347073523, Timestamp('1987-10-02 03:19:21.029038'): 2.2282123763101054, Timestamp('1987-10-18 03:19:38.263006'): 2.052230389794686, Timestamp('1987-11-19 03:20:17.361050'): None, Timestamp('1987-02-11 03:18:33.055063'): -18.70279996894706, Timestamp('1987-03-31 03:20:21.254000'): -5.275542830187344, Timestamp('1987-09-23 03:24:56.922019'): 0.8125911445736622, Timestamp('1987-10-09 03:25:16.189019'): -18.49079920191336, Timestamp('1987-02-11 03:18:56.795081'): -22.5052194068006, Timestamp('1987-02-27 03:19:34.318000'): -8.035610648370026, Timestamp('1987-03-31 03:20:44.997044'): -5.048465362514538, Timestamp('1987-09-23 03:25:20.687088'): None, Timestamp('1987-10-09 03:25:39.814031'): None, Timestamp('1987-02-02 03:24:46.164094'): -14.38057945049292, Timestamp('1987-03-22 03:26:36.740088'): -21.510586765790634, Timestamp('1987-05-25 03:28:34.133000'): 1.99554741495954, Timestamp('1987-06-26 03:29:17.437094'): -1.9484985479628487, Timestamp('1987-07-12 03:29:36.241000'): 10.482010840406327}

In [52]:
all_landsat

{Timestamp('2013-04-01 03:36:33.783319'): -14.321932489351575}

In [13]:
lst_landsat = {}

for year in range(landsat_start_year, end_year):
   lst_landsat[year] = {}
   for month in range(openwater_month_start, openwater_month_end):
      start_date = ee.Date(f"{year}-{month}-1")
      end_date = ee.Date(f"{year}-{(month+1)}-1")
      
      result = get_average_lst_landsat(landsats_collection, start_date, end_date)
      if(result is None):
         continue
      lst_landsat[year][month] = result
   
   lst_landsat[year] = np.mean(list(lst_landsat[year].values()))
   if lst_landsat[year] is None:
      del lst_landsat[year]

lst_landsat

{1996: 7.615046916830711,
 1997: 6.4652446881016505,
 1998: 8.29735257295523,
 1999: 7.466276454270664,
 2000: 8.444648433179971,
 2001: 6.8573912127145835,
 2002: 11.29452174088983,
 2003: 8.133507938438473,
 2004: 7.756460249341295,
 2005: 8.815491983317168,
 2006: 6.796633766422254,
 2007: 9.983745663150753,
 2008: 8.444456981800055,
 2009: 7.979558202903978,
 2010: 7.409646087737466,
 2011: 7.802575171575934,
 2012: 8.998809616805136,
 2013: 9.029895691880107,
 2014: 10.268777774431134,
 2015: 10.722652392625395,
 2016: 10.075703142279956,
 2017: 9.112241223415538,
 2018: 9.1208041147658,
 2019: 9.838905106835822,
 2020: 10.8137280883284,
 2021: 8.059809265577483,
 2022: 8.255228706723798,
 2023: 9.533078486705547,
 2024: 9.525957321098034}

In [ ]:
del lst_landsat[1983]
del lst_landsat[1984]
del lst_landsat[1985]

In [ ]:
lst_landsat_s = { 1996: 7.615046916830711,
 1997: 6.4652446881016505,
 1998: 8.29735257295523,
 1999: 7.466276454270664,
 2000: 8.444648433179971,
 2001: 6.8573912127145835,
 2002: 11.29452174088983,
 2003: 8.133507938438473,
 2004: 7.756460249341295,
 2005: 8.815491983317168,
 2006: 6.796633766422254,
 2007: 9.983745663150753,
 2008: 8.444456981800055,
 2009: 7.979558202903978,
 2010: 7.409646087737466,
 2011: 7.802575171575934,
 2012: 8.998809616805136,
 2013: 9.029895691880107,
 2014: 10.268777774431134,
 2015: 10.722652392625395,
 2016: 10.075703142279956,
 2017: 9.112241223415538,
 2018: 9.1208041147658,
 2019: 9.838905106835822,
 2020: 10.8137280883284,
 2021: 8.059809265577483,
 2022: 8.255228706723798,
 2023: 9.533078486705547,
 2024: 9.525957321098034}

# MODIS

In [7]:
def M11_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_Day_1km', 'LST_Night_1km']).multiply(0.02).subtract(273.15).rename(['LST_Day', 'LST_Night'])
   return modis_image.addBands(srcImg=scaled_lsts)

def M21_scale_to_celsius(modis_image: ee.Image):
   scaled_lsts = modis_image.select(['LST_Day', 'LST_Night']).subtract(273.15)
   return modis_image.addBands(srcImg=scaled_lsts, overwrite=True)

m11_terra = ee.ImageCollection('MODIS/061/MOD11A2').map(lambda img: M11_scale_to_celsius(img))
m21_terra = ee.ImageCollection('MODIS/061/MOD21C2').map(lambda img: M21_scale_to_celsius(img))
m11_aqua = ee.ImageCollection('MODIS/061/MYD11A2').map(lambda img: M11_scale_to_celsius(img))
m21_aqua = ee.ImageCollection('MODIS/061/MYD21C2').map(lambda img: M21_scale_to_celsius(img))

m11_coll = m11_terra.merge(m11_aqua)
m21_coll = m21_terra.merge(m21_aqua)

In [8]:
def get_average_lst_modis(modis_coll: ee.ImageCollection, start_time, end_time):
   if(modis_coll.size() == 0):
      return None
   mean_temp = modis_coll.filterDate(start_time, end_time).mean()
   day_night_temp = mean_temp.expression("(b('LST_Day') + b('LST_Night')) / 2").rename('LST')
   temp = day_night_temp.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return temp.getInfo()['LST']

In [9]:
lst_m21 = {}
lst_m11 = {}

for year in range(modis_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result_m11 = get_average_lst_modis(m11_coll, start_date, end_date)
   # print(f"Complete M11 in {start_date} -- {end_date}")
   result_m21 = get_average_lst_modis(m21_coll, start_date, end_date)
   # print(f"Complete M21 in {start_date} -- {end_date}")
   if(result_m11 is not None):
      lst_m11[year] = result_m11
   if(result_m21 is not None):
      lst_m21[year] = result_m21

print(lst_m11, lst_m21)

{2001: 8.518391334434845, 2002: 10.562313431334651, 2003: 7.857616372707389, 2004: 7.395629688897221, 2005: 9.14143065926696, 2006: 7.007350175325124, 2007: 9.94451399113264, 2008: 8.258408651177756, 2009: 8.258186946255545, 2010: 7.098301657039377, 2011: 8.083154459303568, 2012: 9.068294740221727, 2013: 8.22769966377824, 2014: 10.313111054749466, 2015: 10.214666106018871, 2016: 10.278019374073171, 2017: 9.111631967910679, 2018: 9.083092106624825, 2019: 9.750912104506595, 2020: 10.673629396456128, 2021: 8.178589646995642, 2022: 8.422860681650373, 2023: 10.092309299156067, 2024: 9.718802938390988} {2001: 6.523447096502005, 2002: 11.087026927344537, 2003: 8.227190985636412, 2004: 7.918499190633716, 2005: 9.834607190557838, 2006: 7.375761008911375, 2007: 10.603713281690437, 2008: 8.734014051703918, 2009: 8.932528225671893, 2010: 7.71452877253441, 2011: 8.775993993759588, 2012: 9.776709236956195, 2013: 8.83295322142326, 2014: 10.90875064506874, 2015: 11.124346848601931, 2016: 11.0724993132

In [10]:
lst_m11

{2001: 8.518391334434845,
 2002: 10.562313431334651,
 2003: 7.857616372707389,
 2004: 7.395629688897221,
 2005: 9.14143065926696,
 2006: 7.007350175325124,
 2007: 9.94451399113264,
 2008: 8.258408651177756,
 2009: 8.258186946255545,
 2010: 7.098301657039377,
 2011: 8.083154459303568,
 2012: 9.068294740221727,
 2013: 8.22769966377824,
 2014: 10.313111054749466,
 2015: 10.214666106018871,
 2016: 10.278019374073171,
 2017: 9.111631967910679,
 2018: 9.083092106624825,
 2019: 9.750912104506595,
 2020: 10.673629396456128,
 2021: 8.178589646995642,
 2022: 8.422860681650373,
 2023: 10.092309299156067,
 2024: 9.718802938390988}

In [11]:
lst_m21

{2001: 6.523447096502005,
 2002: 11.087026927344537,
 2003: 8.227190985636412,
 2004: 7.918499190633716,
 2005: 9.834607190557838,
 2006: 7.375761008911375,
 2007: 10.603713281690437,
 2008: 8.734014051703918,
 2009: 8.932528225671893,
 2010: 7.71452877253441,
 2011: 8.775993993759588,
 2012: 9.776709236956195,
 2013: 8.83295322142326,
 2014: 10.90875064506874,
 2015: 11.124346848601931,
 2016: 11.072499313254557,
 2017: 9.697003172751204,
 2018: 9.721762029880114,
 2019: 10.577461115660952,
 2020: 11.42118506104802,
 2021: 8.943680470222272,
 2022: 9.072915370205486,
 2023: 11.284890175973027,
 2024: 10.757315581774394}

In [ ]:
lst_m11_s = {2001: 8.518391334434845,
 2002: 10.562313431334651,
 2003: 7.857616372707389,
 2004: 7.395629688897221,
 2005: 9.14143065926696,
 2006: 7.007350175325124,
 2007: 9.94451399113264,
 2008: 8.258408651177756,
 2009: 8.258186946255545,
 2010: 7.098301657039377,
 2011: 8.083154459303568,
 2012: 9.068294740221727,
 2013: 8.22769966377824,
 2014: 10.313111054749466,
 2015: 10.214666106018871,
 2016: 10.278019374073171,
 2017: 9.111631967910679,
 2018: 9.083092106624825,
 2019: 9.750912104506595,
 2020: 10.673629396456128,
 2021: 8.178589646995642,
 2022: 8.422860681650373,
 2023: 10.092309299156067,
 2024: 9.718802938390988}

lst_m21_s = {2001: 6.523447096502005,
 2002: 11.087026927344537,
 2003: 8.227190985636412,
 2004: 7.918499190633716,
 2005: 9.834607190557838,
 2006: 7.375761008911375,
 2007: 10.603713281690437,
 2008: 8.734014051703918,
 2009: 8.932528225671893,
 2010: 7.71452877253441,
 2011: 8.775993993759588,
 2012: 9.776709236956195,
 2013: 8.83295322142326,
 2014: 10.90875064506874,
 2015: 11.124346848601931,
 2016: 11.072499313254557,
 2017: 9.697003172751204,
 2018: 9.721762029880114,
 2019: 10.577461115660952,
 2020: 11.42118506104802,
 2021: 8.943680470222272,
 2022: 9.072915370205486,
 2023: 11.284890175973027,
 2024: 10.757315581774394}

# ERA5

In [15]:
def ERA5_scale_to_celsius(era5_image: ee.Image):
   scaled_lsts = era5_image.select(["mean_2m_air_temperature"]).subtract(273.15).rename(["LST"])
   return era5_image.addBands(srcImg=scaled_lsts, overwrite=True)

era5_coll = ee.ImageCollection("ECMWF/ERA5/MONTHLY").map(lambda img: ERA5_scale_to_celsius(img))

In [16]:
def get_average_lst_era5(era5_coll: ee.ImageCollection, start_time, end_time):
   if(era5_coll.size() == 0):
      return None
   mean_temp = era5_coll.filterDate(start_time, end_time).mean()
   temp = mean_temp.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return temp.getInfo()['LST']

In [17]:
lst_era5 = {}

for year in range(landsat_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result = get_average_lst_era5(era5_coll, start_date, end_date)
   if(result is not None):
      lst_era5[year] = result
      
lst_era5

{1996: 9.450155880882175,
 1997: 10.306804804710536,
 1998: 10.124025263851733,
 1999: 9.953970489104826,
 2000: 10.897080923980255,
 2001: 11.237577393579986,
 2002: 11.02499017361202,
 2003: 10.489261758126426,
 2004: 10.945637833437745,
 2005: 11.244417451457139,
 2006: 10.083577737021876,
 2007: 11.765040295289413,
 2008: 11.061690752218151,
 2009: 10.35989784117505,
 2010: 10.440376515999297,
 2011: 11.403798160090886,
 2012: 10.77953644155891,
 2013: 9.971784275350128,
 2014: 11.108825121667898,
 2015: 12.168882374570854,
 2016: 11.078814723490853,
 2017: 10.931184669283283,
 2018: 11.564613195835445,
 2019: 11.313782876770295,
 2020: 10.744542470002987}

In [ ]:
lst_era5_s = { 1996: 9.450155880882175,
 1997: 10.306804804710536,
 1998: 10.124025263851733,
 1999: 9.953970489104826,
 2000: 10.897080923980255,
 2001: 11.237577393579986,
 2002: 11.02499017361202,
 2003: 10.489261758126426,
 2004: 10.945637833437745,
 2005: 11.244417451457139,
 2006: 10.083577737021876,
 2007: 11.765040295289413,
 2008: 11.061690752218151,
 2009: 10.35989784117505,
 2010: 10.440376515999297,
 2011: 11.403798160090886,
 2012: 10.77953644155891,
 2013: 9.971784275350128,
 2014: 11.108825121667898,
 2015: 12.168882374570854,
 2016: 11.078814723490853,
 2017: 10.931184669283283,
 2018: 11.564613195835445,
 2019: 11.313782876770295,
 2020: 10.744542470002987 }